In [ ]:
import matplotlib.pyplot as plt
from binance import Client    #to connect to binance API
import pandas as pd   
import numpy as np
import matplotlib.animation as animation    #for visualizing an interactive chart
from matplotlib.dates import DateFormatter

In [ ]:
#you need this, otherwise the interactive plot doesn't work inside Jupyter Notebook (no need for this in IDE)
%matplotlib notebook  

#to connect to Binance API (log in to Binance - API Management - Create API - from there you get API and secret key)
#https://www.binance.com/en/support/faq/how-to-download-and-set-up-binance-code-api-af014f44f45845debf79b4cf81333a25
API = Client('secret key1', 'secret key2')

crypto = 'BTCUSDT'

In [ ]:
#https://python-binance.readthedocs.io/en/latest/binance.html?highlight=get_historical_klines#binance.client.AsyncClient.get_historical_klines

def crypto_prices(crypto, frequency, past):
    df = pd.DataFrame(API.get_historical_klines(crypto, frequency, past))     
    df.columns = ['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore']     
    df = df.set_index('Open Time')
    df.index = pd.to_datetime(df.index, unit='ms') 
    df = df.drop(['Open','High','Low','Volume','Close Time','Quote Asset Volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore'], axis=1) 
    df = df.astype('float')
    
    #define BTC that is owned (info from part1)
    df['BTC_Own'] = 0.041475
    
    df['USD_Owned'] = np.NaN
    df['USD_Owned'] = df['BTC_Own'].multiply(df['Close'])
    
    #total amount invested (info from part1)
    df['Total_Paid'] = np.NaN
    df['Total_Paid'] = 1654
    
    df['Gain_Loss_in_USD'] = np.NaN
    df['Gain_Loss_in_USD'] = df['USD_Owned'] - (df['Total_Paid'])
    
    df['Gain_Loss_Pct'] = np.NaN
    df['Gain_Loss_Pct'] = df['USD_Owned'] / (df['Total_Paid'])
    
    df['Pct. Change (Gain/Loss)'] = np.NaN
    df['Pct. Change (Gain/Loss)'] = df['Gain_Loss_in_USD'].pct_change() 
    
    df['USD_Change'] = np.NaN
    df['USD_Change'] = df['Pct. Change (Gain/Loss)']
    df['USD_Change'] = df['USD_Change'].shift(periods=-1, axis='index')
    df['USD_Change'] = df['Gain_Loss_in_USD'].multiply(df['USD_Change'])
    df['USD_Change'] = df['USD_Change'].shift(periods=1, axis='index')
    df['USD_Change'] = df['USD_Change'].round(2)
    
    df['BTC_Price_Even'] = np.NaN
    df['BTC_Price_Even'] = df['Total_Paid'] / df['BTC_Own']
    
    df['Cum_Ret_BTC'] = np.NaN                
    df['Cum_Ret_BTC'] = ((df['Close'].pct_change()) + 1).cumprod() - 1
         
    return df


In [ ]:
#for graphical representation (UTC time zone)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(8, 5))

def animate(i):
    data = crypto_prices(crypto,'1m','7.2e+6')    #define the frequency (1 min) and past time (here it is set to 2h, check docs - link above)
    ax1.clear()   #to clear the axes and prevent charts overlaying
    ax1.plot(data.index, data.Close, color='xkcd:black')
    ax1.xaxis.set_major_formatter(DateFormatter('%H:%M'))
    ax1.set_title('BTC Price ($)')
    plt.gcf().autofmt_xdate()  #this is to rotate the tick labels on the x-axis (https://www.geeksforgeeks.org/matplotlib-figure-figure-autofmt_xdate-in-python/)
    plt.tight_layout()
    
    ax2.clear()
    ax2.plot(data.index, data.Gain_Loss_in_USD, color='xkcd:blue')
    ax2.xaxis.set_major_formatter(DateFormatter('%H:%M'))
    ax2.set_title('Profit or Loss ($)')
    plt.gcf().autofmt_xdate()  
    plt.tight_layout()    

    ax3.clear()
    ax3.plot(data.index, data.USD_Owned, color='xkcd:orange')
    ax3.set(xlabel='Time (hh:mm)')
    ax3.xaxis.set_major_formatter(DateFormatter('%H:%M'))
    ax3.set_title('Actual $ Own')
    plt.gcf().autofmt_xdate()  
    plt.tight_layout() 

    ax4.clear()
    ax4.plot(data.index, data.USD_Change)
    ax4.set(xlabel='Time (hh:mm)')
    ax4.xaxis.set_major_formatter(DateFormatter('%H:%M'))
    ax4.set_title('$ Change')
    plt.gcf().autofmt_xdate()  
    plt.tight_layout() 

ani = animation.FuncAnimation(fig, animate, interval=1000)   #interval is in ms (so in this case it will update every sec)

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))

def animate(i):
    data = crypto_prices(crypto,'1m','7.2e+6')
    ax1.clear()
    ax1.plot(data.index, data.Gain_Loss_Pct, color='xkcd:greyish blue')
    ax1.set(xlabel='Time (hh:mm)')
    ax1.xaxis.set_major_formatter(DateFormatter('%H:%M'))
    ax1.set_title('Profit/loss in pct. for a total investment')
    plt.gcf().autofmt_xdate()  
    plt.tight_layout() 

    ax2.clear()
    ax2.boxplot(data.Gain_Loss_Pct)
    ax2.set_title('Boxplot of profit/loss in pct.')
    ax2.set_xticks(ticks=[1], labels=['y=1 (even), y<1 (loss), y>1 (profit)'])
    ax2.yaxis.grid(True, linestyle='-', which='major', color='xkcd:light grey',
               alpha=0.5)  
    plt.tight_layout()     
    

ani = animation.FuncAnimation(fig, animate, interval=1000)  

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))

def animate(i):
    data = crypto_prices(crypto,'1m','7.2e+6')
    ax1.clear()
    ax1.hist(data.USD_Change, rwidth=0.5, bins=1000, linewidth=0.5, edgecolor='xkcd:army green')
    ax1.set_title('Distribution of $ Change every second')
    ax1.set(xlabel='USD', ylabel='Frequency')
    plt.gcf().autofmt_xdate()
    plt.tight_layout()
       
    ax2.clear()
    ax2.plot(data.index, data.Cum_Ret_BTC, color='xkcd:light olive')
    ax2.set(xlabel='Time (hh:mm)')
    ax2.xaxis.set_major_formatter(DateFormatter('%H:%M'))
    ax2.set_title('Cumulative return of BTC in the last 2 hours')
    plt.gcf().autofmt_xdate()  
    plt.tight_layout() 

ani = animation.FuncAnimation(fig, animate, interval=1000)   

plt.show()
